In [6]:
% cd "../"

/home/akali/projects


In [7]:
% cd "codenames"

/home/akali/projects/codenames


In [1]:
import logging
from time import sleep

from codenames.game import QuitGame
from playground.boards import *  # noqa
from utils import configure_logging

configure_logging()

log = logging.getLogger(__name__)

Logging configured


In [3]:
from codenames.game import DEFAULT_MODEL_ADAPTER
from codenames.game.runner import GameRunner
from codenames.utils.loader.model_loader import ModelIdentifier, load_model_async
from codenames.utils.model_adapters import HEBREW_SUFFIX_ADAPTER
from codenames.solvers import NaiveHinter, NaiveGuesser

# model_id = ModelIdentifier("english", "wiki-50", False)
# model_id = ModelIdentifier("english", "google-300", False)
# model_id = ModelIdentifier("hebrew", "twitter", False)
# model_id = ModelIdentifier("hebrew", "ft-200", False)
# model_id = ModelIdentifier(language="hebrew", model_name="skv-ft-150", is_stemmed=True)
model_id = ModelIdentifier(language="hebrew", model_name="skv-cbow-30", is_stemmed=True)
# model_id = ModelIdentifier("hebrew", "skv-cbow-150", True)

# os.environ[MODEL_NAME_ENV_KEY] = model_id.model_name
# os.environ[IS_STEMMED_ENV_KEY] = "1" if model_id.is_stemmed else ""
adapter = HEBREW_SUFFIX_ADAPTER if model_id.language == "hebrew" and model_id.is_stemmed else DEFAULT_MODEL_ADAPTER
load_model_async(model_id)

blue_hinter = NaiveHinter("Leonardo", model_identifier=model_id, model_adapter=adapter)
blue_guesser = NaiveGuesser("Bard", model_identifier=model_id, model_adapter=adapter)
red_hinter = NaiveHinter("Adam", model_identifier=model_id, model_adapter=adapter)
red_guesser = NaiveGuesser("Eve", model_identifier=model_id, model_adapter=adapter)

game_runner = GameRunner(blue_hinter, red_hinter, blue_guesser, red_guesser)

Loading model language='hebrew' model_name='skv-cbow-30' is_stemmed=True
Loading model... {'model': ModelIdentifier(language='hebrew', model_name='skv-cbow-30', is_stemmed=True)}
loading KeyedVectors object from /home/akali/.cache/language_data/hebrew/skv-cbow-30/model.kv
{'uri': '/home/akali/.cache/language_data/hebrew/skv-cbow-30/model.kv', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}


In [6]:
# game_runner.run_game(language="english", board=english_board)
game_runner.run_game(language="hebrew", board=HEBREW_BOARD_1)

Loading model language='hebrew' model_name='skv-ft-150' is_stemmed=True
Loading model language='hebrew' model_name='skv-ft-150' is_stemmed=True
Loading model language='hebrew' model_name='skv-ft-150' is_stemmed=True
Loading model language='hebrew' model_name='skv-ft-150' is_stemmed=True

-----
[Blue] turn.
Creating proposals for group size [4]...
Waiting for task manager to finish...
Creating proposals for group size [3]...
Waiting for task manager to finish...
Creating proposals for group size [2]...
Waiting for task manager to finish...
Creating proposals for group size [1]...
Waiting for task manager to finish...
Got 38 proposals.
Best 5 proposals: 
('קברנ', 'אדמ') = ('המת', 6.76)
('שלמ', 'יקומ') = ('אינסוף', 6.49)
('קברנ', 'זכוכית') = ('סרקופג', 6.49)
('קברנ',) = ('קבור', 6.29)
('שלמ', 'דרור') = ('שלמון', 6.28)
Picked proposal: Proposal(word_group=('קברנ', 'אדמ'), hint_word=המת, hint_word_frequency=0.939, distance_group=0.313, distance_gray=0.490, distance_opponent=0.399, distance_

Winner(team_color=<TeamColor.RED: 'Red'>, reason=<WinningReason.TARGET_SCORE_REACHED: 'Target score reached'>)

In [ ]:
from codenames.online import NamecodingGameRunner, NamecodingLanguage

online_runner = None
try:
    online_runner = NamecodingGameRunner(blue_hinter, red_hinter, blue_guesser, red_guesser)
    online_runner.auto_start(language=NamecodingLanguage.ENGLISH, clock=False)
    sleep(1)
except QuitGame:
    pass
except Exception as e:  # noqa
    log.exception("Error occurred")
finally:
    if online_runner is not None:
        log.info(f"Winner: {online_runner.winner}")
        online_runner.close()
log.info("Done")

In [6]:
from codenames.utils.loader.model_loader import load_model
from codenames.boards.hebrew import *  # noqa

# for word in HEBREW_WORDS:
#     for other in HEBREW_WORDS:
#         if word != other and editdistance.eval(word, other) <= 2 and abs(len(word) - len(other)) >= 2:
#             print(f"{word} {other}")

print(f"Got {len(HEBREW_WORDS)} words")
model = load_model(model_id)
errors = []
for word in HEBREW_WORDS:
    # log.info("Checking %s", word)
    try:
        x = model[adapter.to_model_format(word)]
    except Exception as e:
        errors.append(word)
print("Done with %d errors" % len(errors))

Got 424 words
Loading model language='hebrew' model_name='skv-cbow-30' is_stemmed=True
Done with 0 errors
